BoxPlots for All patients

In [1]:
#Import Modules

import sys
sys.path.append("..") #give this script access to all modules in parent directory
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import matplotlib.lines as mlines
from scipy.io import loadmat
import scipy
from scipy.stats import skew
from Data_Cleaning import preprocess
from Data_Cleaning import preprocess_sickbay
import Filtering
import Actigraph_Metrics

C:\Users\jakes\AppData\Local\Temp\ipykernel_1272\888196890.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Define Functions

Load All Patient Data

In [21]:
def fillArrays(signal, Mean, AUC, PEAK_PEAK, Stdev): 
    x = []
    for i in range(len(signal)):
        x.append(i) #x is the x axis for signal data, it is necessary for AUC calculations. Signal is ordered as is x, so this works
    Mean.append(np.mean(signal))
    AUC.append(Actigraph_Metrics.calc_area_under_curve(x,signal))
    PEAK_PEAK.append(max(signal)-min(signal))
    Stdev.append(np.std(signal))

def fill_SBS_Changes(vitals_SBS,SBS_Changes,i):
    state = vitals_SBS[i+1]-vitals_SBS[i]
    if(state > 0):
        SBS_Changes.append(1) #1 signifies increase in SBS
    elif state == 0: 
        SBS_Changes.append(0)
    else:
        SBS_Changes.append(-1)


In [4]:
#Patients with -2 or 2 are ['Patient13', 'Patient2', 'Patient3', 'Patient4', 'Patient9']
data_dir = r'C:\Users\jakes\Documents\DT 6 Analysis\PythonCode\PedAccel\data_analysis\PythonPipeline\PatientData\2_neg2'
window_size = 100 #100 is 1 second worth of time
lead_time = 10
slice_size_min = 15
sr = .5

In [24]:

#There is no error handling in place, the .mat file must exist
for patient in os.listdir(data_dir):
    # filter out non-directories
    patient_dir = os.path.join(data_dir, patient)
    if os.path.isdir(patient_dir):
       # data_filepath_accel = os.path.join(patient_dir, f'{patient}_{lead_time}MIN_{slice_size_min - lead_time}MIN.mat')           
        data_filepath_vitals = os.path.join(patient_dir, f'{patient}_SICKBAY_{slice_size_min - lead_time}MIN_{lead_time}MIN.mat')
        
       # accel_data = loadmat(data_filepath_accel)
       # x_mag = accel_data["x_mag"]
       # accel_SBS = accel_data["sbs"].flatten()
        
        vitals_data = loadmat(data_filepath_vitals)
        temp_hr = vitals_data['heart_rate']
        temp_SpO2 = vitals_data['SpO2']
        temp_rr = vitals_data['respiratory_rate']

        vitals_SBS = vitals_data['sbs'].flatten()
        hr = []
        rr = []
        SpO2 = []
        bpm = []
        bps = []
        bpd = []
        vitals_list = [hr,rr,SpO2]
        vitals_names = ['hr','rr','spo2']
        temp_vitals = [temp_hr,temp_rr, temp_SpO2] 
        
        flag_list = [0] * (int)(sr * 60 * slice_size_min) #generate a list to insert in place of invalid data, 
        #this list serves as a flag for a window to ignore in the box plot function

        for j in range(len(vitals_list)): #go through every vitals metric
            #print(f'original {vitals_names[j]} vitals array shape: {np.array(temp_vitals[j]).shape} ')
            for i in range(len(vitals_SBS)): #go through every SBS score for each vitals metric
                if (Filtering.checkVitals(temp_vitals[j][i], slice_size_min, vitals_names[j])): #check the data in a single window
                    vitals_list[j].append(temp_vitals[j][i]) #append that single window data to the 2D hr,rr,spo2,bpm,bps,bpd arrays if that window's data is valid
                else:
                    vitals_list[j].append(flag_list) #append an array of zeros for window number i for the jth vitals metric if the data is invalid(i.e. too many NaN points)
                   # print(f'{vitals_names[j]} SBS index {i} has insufficient data, zeros appended in place') 
            #print(f'final {vitals_names[j]} vitals array shape: {np.array(vitals_list[j]).shape}') #should be the number of SBS scores by the number of samples in a window


        j = 0

        for k in  range(len(vitals_list)): 
            
            cleared_SBS = []
            AUC = []
            PEAK_PEAK = []
            Mean = []
            Stdev = []

            signal_2D = (np.array(vitals_list[k])) #signal_2D is a single vitals metric

            for j in range(len(signal_2D)):
                if(np.mean(signal_2D[j])!=0): #check that neither is a flag list, signal_2D[j] is a single window for a single metric
                #define arrays to populate on each iteration
                    # Populate array with changes in SBS
                    cleared_SBS.append(vitals_SBS[j])
                    fillArrays(signal_2D[j], Mean, AUC, PEAK_PEAK, Stdev)
                #else:
                    #print(f'flag list detected at index {j}')

            i = 0
            
            assert(len(Mean)== len(AUC))
            assert(len(AUC) == len(PEAK_PEAK))
            assert(len(cleared_SBS)==len(Mean))
            assert(len(vitals_SBS) >= len(cleared_SBS))


            average_change_in_mean_2 = []
            average_change_in_AUC_2 = []
            average_change_in_Peaks_2 = []
            average_change_in_Stdev_2 = []

            average_change_in_mean_0 = []
            average_change_in_AUC_0 = []
            average_change_in_Peaks_0 = []
            average_change_in_Stdev_0 = []


            if(len(cleared_SBS) > 2):
                for i in range(len(cleared_SBS)-1):
                    if((cleared_SBS[i] == 0) and cleared_SBS[i+1] == 2):
                        print(f'The SBS score at index {i} is {cleared_SBS[i]} and the SBS score at index {i + 1} is {cleared_SBS[i+1]}')
                        print(f'The change in PEAK_PEAK values from {cleared_SBS[i]} to {cleared_SBS[i+1]} is {PEAK_PEAK[i+1]-PEAK_PEAK[i]}')
                        print(f'The change in AUC from {cleared_SBS[i]} to {cleared_SBS[i+1]} is {AUC[i+1]-AUC[i]}')
                        print(f'The change in Mean values from {cleared_SBS[i]} to {cleared_SBS[i+1]} is {Mean[i+1]-Mean[i]}')
                        print(f'The change in Stdev values from {cleared_SBS[i]} to {cleared_SBS[i+1]} is {Stdev[i+1]-Stdev[i]}')

                        average_change_in_mean_2.append(abs(Mean[i+1]-Mean[i]))
                        average_change_in_AUC_2.append(abs(AUC[i+1]-AUC[i]))
                        average_change_in_Peaks_2.append(abs(PEAK_PEAK[i+1]-PEAK_PEAK[i]))
                        average_change_in_Stdev_2.append(abs(Stdev[i+1]-Stdev[i]))

                    elif((cleared_SBS[i] == 0) and (cleared_SBS[i+1] == 1)):
                        average_change_in_mean_0.append(abs(Mean[i+1]-Mean[i]))
                        average_change_in_AUC_0.append(abs(AUC[i+1]-AUC[i]))
                        average_change_in_Peaks_0.append(abs(PEAK_PEAK[i+1]-PEAK_PEAK[i]))
                        average_change_in_Stdev_0.append(abs(Stdev[i+1]-Stdev[i]))

                print('_____________________________________________________________________________________________________')
                print(f'******** SUMMARY {vitals_names[k]} {patient}*********\n')
                print(f'The average change in mean from SBS of 0 to 2 was {np.mean(average_change_in_mean_2)}')
                print(f'The average change in AUC from SBS of 0 to 2 was {np.mean(average_change_in_AUC_2)}')
                print(f'The average change in Peak to Peak from SBS 0 to 2 was {np.mean(average_change_in_Peaks_2)}')
                print(f'The average change in Stdev from SBS 0 to 2 was {np.mean(average_change_in_Stdev_2)}\n')

                print(f'The average change in mean from SBS 0 to 1 was {np.mean(average_change_in_mean_0)}')
                print(f'The average change in AUC from SBS 0 to 1  was {np.mean(average_change_in_AUC_0)}')
                print(f'The average change in Peak to Peak from SBS 0 to 1 was {np.mean(average_change_in_Peaks_0)}')
                print(f'The average change in Stdev from SBS 0 to 1 was {np.mean(average_change_in_Stdev_0)}')

                print('_____________________________________________________________________________________________________')



The SBS score at index 10 is 0 and the SBS score at index 11 is 2
The change in PEAK_PEAK values from 0 to 2 is 34.0
The change in AUC from 0 to 2 is 23731.0
The change in Mean values from 0 to 2 is 52.84222222222223
The change in Stdev values from 0 to 2 is 14.010503317627007
The SBS score at index 15 is 0 and the SBS score at index 16 is 2
The change in PEAK_PEAK values from 0 to 2 is 55.0
The change in AUC from 0 to 2 is 18252.5
The change in Mean values from 0 to 2 is 40.57111111111112
The change in Stdev values from 0 to 2 is 15.18759618059027
The SBS score at index 19 is 0 and the SBS score at index 20 is 2
The change in PEAK_PEAK values from 0 to 2 is 41.0
The change in AUC from 0 to 2 is 8118.5
The change in Mean values from 0 to 2 is 18.071111111111108
The change in Stdev values from 0 to 2 is 13.512691255680787
_____________________________________________________________________________________________________
******** SUMMARY hr Patient13*********

The average change in mea